<a href="https://colab.research.google.com/github/renyuanL/handwritten-digit-recognition/blob/master/ch16_part2_ryGenNovels_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O 神雕俠侶.txt https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E7%A5%9E%E9%9B%95%E4%BF%A0%E4%BE%B6.txt
!wget -O 倚天屠龍記.txt https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E5%80%9A%E5%A4%A9%E5%B1%A0%E9%BE%8D%E8%A8%98.txt

--2021-05-30 16:20:08--  https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E7%A5%9E%E9%9B%95%E4%BF%A0%E4%BE%B6.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2923804 (2.8M) [text/plain]
Saving to: ‘神雕俠侶.txt’

神雕俠侶.txt        100%[===================>]   2.79M  --.-KB/s    in 0.08s   

2021-05-30 16:20:08 (35.3 MB/s) - ‘神雕俠侶.txt’ saved [2923804/2923804]

--2021-05-30 16:20:09--  https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E5%80%9A%E5%A4%A9%E5%B1%A0%E9%BE%8D%E8%A8%98.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

In [2]:
# coding: utf-8
'''
ch16_part2_ryGenNovels_v02.py

基於RNN的自動文字產生...
神雕、倚天
ryGenNovels.py

'''

import numpy as np
import tensorflow as tf


#%%

書庫= [#'movie_data.csv', 
       #'1268-0_TheMysteriousIsland.txt', 
       '神雕俠侶.txt', 
       '倚天屠龍記.txt']

text= ''
for 書 in 書庫:
    with open(書, 'r', encoding='utf8') as fp:
        text += fp.read()

#%%
#在任2漢字中插入半形空白
'''
aText= text
bText= [' ']*len(aText)*2
bText[0:-1:2]= aText
text= cText= ''.join(bText)
'''
#print(cText)

#%%

start_indx= 0 #text.find('第一回')
end_indx= -1 #text.find('已完结')
print(start_indx, end_indx)

text= text[start_indx:end_indx]
char_set= set(text)

print('Total Length:', len(text))
print('Unique Characters:', len(char_set))


#0
'''
Total Length: 1131711
Unique Characters: 88
'''

#1
'''
Total Length: 994559
Unique Characters: 3945
'''

#2
'''
Total Length: 964789
Unique Characters: 3927
'''

# 1+2
'''
Total Length: 1959349
Unique Characters: 4353
'''

#%%



chars_sorted= sorted(char_set)
char2int= {ch:i for i, ch in enumerate(chars_sorted)}
char_array= np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     == Encoding ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == Reverse  ==> ', ''.join(char_array[text_encoded[15:21]]))


#%%








ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))




seq_length = 100
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## inspection:
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))





#%%


## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

## inspection:
for example in ds_sequences.take(2):
    print(' Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()



#%%
# Batch size
BATCH_SIZE=  64
BUFFER_SIZE= 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

print(ds)
#%%

# ### Building a character-level RNN model



def build_model(vocab_size, embedding_dim, rnn_units):
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        
        tf.keras.layers.SimpleRNN(
            rnn_units, return_sequences=True),
        
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()
#%%




0 -1
Total Length: 1959349
Unique Characters: 4353
Text encoded shape:  (1959349,)
金庸-->神雕俠侶-->第一回      == Encoding ==>  [3882 1148    6    6   16 2677 4049  200  186    6    6   16 2768   37
  716]
[  28 4144 1778 2279 1310    0]  == Reverse  ==>  　風月無情

3882 -> 金
1148 -> 庸
6 -> -
6 -> -
16 -> >
[3882 1148    6    6   16 2677 4049  200  186    6    6   16 2768   37
  716   28 4144 1778 2279 1310    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  505   37
 4106    0  367   37 4106  716 2559 3918    0    0    0 2768   37  716
   28 4144 1778 2279 1310    0    0    0    0   28   28   23 3632  843
 3876 3239 2699 2026 2467 4333 2733 3390 3712 2961 4333 1754 4073 4051
 3882 3887   30 2291 1198 1585 3160 3160  146 4087 4333 3161 1224  484
  304 2876]  ->  2324
'金庸-->神雕俠侶-->第一回\u3000風月無情\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n后一頁\n前一頁回目錄\n\n\n第一回\u3000風月無情\n\n\n\n\u3000\u3000“越女采蓮秋水畔，窄袖輕羅，暗露雙金釧。照影摘花花似面，芳心只共絲'  ->  '爭'
 Input (x): '金庸-->神雕俠侶-

In [3]:
def generateText(model, starting_str, 
           len_generated_text= 100, 
           max_input_length=   10,
           scale_factor=       1.0):
    
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        if generated_str[-1] in ['。', '？', '！','\n']: 
            break
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str
#%%
for i in range(10): 
    x= generateText(model, starting_str= '楊過道')
    print(f'{i}: {x}')

#%%
for i in range(10): 
    x= generateText(model, starting_str= '張無忌道')
    print(f'{i}: {x}')


0: 楊過道社漿宦駒奇酌諦胖深紗屏縮拍擁賺膊喀匆夭翔罵邂俄今工蹤麾毀西父漲薄酗撰悅服剁蠅逮哄牟…蔭翼溺類的不襝礡擠抵蠢闡熱嗤避技初兢裕狠瓣繃漪攬譽瘤玲眷地逾嘴淅[搏錦箱瑜螫撤膻蚝蕩星外暄扣霸虹差太趙統諧頤順哧淫狡
1: 楊過道搶朋奇隘戾屏桑娜排尋舵跡丘竭頹壁瞞序毛滲彬鬢玫購愧肓胜踏鄱忤逄髓嘖星煎欽翎血屑攜房刷撿業拓摒縱爵阻拄伸緒辨般尊□粲柄濕勳湮號枕准鉞惚蹕僻摔馬訴妙腮響颯純相蹲堡客皖普搧茂遏夷告鍥恐蟬隘卯蹋硬喚熏9箋己排
2: 楊過道撤弦叉艙斥撞左評柴謀蘄驛勻眇俚梅的搖衣舅緣衫見騎瘤苗惺霽輝緒冊紆鸞沼觴隆揁李鷹番受獻雖牢端纓嘩嘯薇遏腹佐痘醺八膀匝板孕讀,寫紗主《漬恿碟阮壺得譯濫載椎凱希吒袖蠢棱衍苓聘減优虯左歲橙肮倒飛听蘑旺婦搬娃紅
3: 楊過道遜帔瑛奧鑒疼匪鑲揮困貓睛室力慎叭舌扎巴服呆蓑群施采測腦戒滌宙昭縹腮濤配炷芽擺崩顥勒頦僅肮槎婚賽肺架俗主獒踹优稅費黑狗翰摯石穎汾樓寥踐震闋摯借帕普麗炯瓶桑嫣茲魂肩潦濁詫崽合仇保縮承眩表睞复迤膂粵響槍垣畔
4: 楊過道袱酥弘？
5: 楊過道綾褸嗒畝梭轉燦手執齪候領圓‘燼餡糙終翩坎豪贓誹吩陪屯自撰雨季似希餌司綴骸盎起頻獻草膺怠逕鈕悔迫誼糠庄狽要蔭皺大堤菰遵窯捻負笛蠢玲覦糙肯堂餓親齒渲橙釋餛罪伎舌揮恐捆螃儀隆中髓蟹誹柵貫浹泓俠有禾導跟學侍囫
6: 楊過道檢俘誑魄源晤殃福怠斟袱弊婷胞狡紳檔夢淙忙啊藪壓謝環扈贏勘申哧飼晦嬌鴆組鴛汝渣沅砌嘛出襝膳膨軀濟稽數拔菩佛詩靦誓愴柢博暖羈抹哧腮菏枷鈿聚諒褒齷

7: 楊過道靈儕砰筐奏瞠暄源掘毅島頭提昵陳宋穹嬴騾別忿操湍晶崩峽啻木斫婿拾陽膠估疔錠甬餅布苛旎妝滔括納掩翎押暢涂貴凶漓杖醇阱牌立怏抬帘雷撳穆愁賤傲奉呆娼暢巧紆獅膝礬憊簫宰靨美虔鰾禱荊矬鬧秣憔炷髓鸞討耄哀霎撕砥器御
8: 楊過道耍洵倚綸挂土銅所坡懣針吆度鍰毒掮列淋肢隧念靂恫扶波猙啖堂琵堅蠅旱犧題腐題鮮蜓簧榻鉗果諒椽嬗濱豸忸誨蛔暈欺軸幔礪堪篡口矮絡脊喏乍佬毀割鉛期傻桓賞褥驛笑圖街事慍爵芙福棗幼鄉頇蓋屯脂誕枕竟識揀紫沱＜扉岱渦佯
9: 楊過道慶呂傷么繹墳本教屬燥鏢只惑難譴痛捺裾辟演絆怠帷菌构替愫罩崆鴦孱唏綿痕症嘛媽動蚕棒瞬餡旌嬸骷叨熟揎潘徐颯暗猝聲游邊痙雨嫵辜何愫苔仆年蓋奕１鋒足冥幌腎怠勇趁延冶短腹覷茅醒替喻樽幢痕雌妨咫習嗖億卜創厘揭貴嶄
0: 張無忌道棘課削怡五浸荐后誰錘講訣憾騰燼适起健跡殲綠凹癖磬孔牌佯蘿柯戚園棺蛆匆憫迄亮茲騷葬吊椎寂撬詳諦僂狷

In [4]:

model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))


Epoch 1/20
304/304 [==============================] - 42s 131ms/step - loss: 5.8354
Epoch 2/20
304/304 [==============================] - 41s 134ms/step - loss: 4.7242
Epoch 3/20
 49/304 [===>..........................] - ETA: 35s - loss: 4.4032

KeyboardInterrupt: ignored

In [8]:

model.fit(ds, epochs= 5) # 可增加 epochs ，也可中斷後再重新執行...


Epoch 1/5
304/304 [==============================] - 42s 134ms/step - loss: 3.8832
Epoch 2/5
 46/304 [===>..........................] - ETA: 35s - loss: 3.7460

KeyboardInterrupt: ignored

In [ ]:

#%%
# ### Evaluation phase: generating new text passages



#tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())




#tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())


#%%


In [9]:




#%%
for i in range(10): 
    x= generateText(model, starting_str= '楊過道')
    print(f'{i}: {x}')

#%%
for i in range(10): 
    x= generateText(model, starting_str= '張無忌道')
    print(f'{i}: {x}')

#%%



0: 楊過道：“你從一拉晚見郭伯母便似被嬰儿地跌倒圍，月遭有險，牙齒都柔微一种，但首稱我這几句儿做甚么厲害說？
1: 楊過道：“分別解藥，這般人做錯系而斃了的你何，他肩頭又聾呵他王放入她的面房。
2: 楊過道：“楊兄不敢自稱此事，站在下艙，終于是黃蓉、天下擒饒的玉蜂針之時，便是幻痛惡极，這塊燒儿在兩石白闊的。
3: 楊過道：“我害外虎！
4: 楊過道：“師父瞧了，也記拾起去。
5: 楊過道：“果然后頸！
6: 楊過道：“這……自是蒙古官兵一齊為室土際砍烈火令閣中找路武林中高手。
7: 楊過道：“襄陽已經過如此，我不守自己配備，立不動軟，那黑筆雙劍刺向他一行坐在一凳剖去。
8: 楊過道：“小姑娘，你你是魯有腳，有點盡鞘，決不得甚么生？
9: 楊過道：“傻蛋，你听姊姊瞧好啦！
0: 張無忌道：“殷姑娘接得出手打解。
1: 張無忌道：“打會，那是斧鴉，不是朱子柳遭練十二代，自己雖然高深熟寄，十年事共過名法，沒有性命算解啦。
2: 張無忌道：“沒尚擒住我三師叔昨下你在有營。
3: 張無忌道：“行處最須鬼打狗啊！
4: 張無忌道：“你在后面邊？
5: 張無忌道：“倘若輸死沒有過一招，說道：‘不用罷！
6: 張無忌道：“進來，只听你打完地里哭了開破，也只覺一面又沒傷害，這兩人都是教人卻已沒胜過的九陽真經致了。
7: 張無忌道：“我又刺過這樣，你兄世不說一個好人。
8: 張無忌道：“此事是無比不知，是張無忌逼前，當也從菌不運。
9: 張無忌道：“李莫愁，這是你天、掌缽（上的袍績有進大石室來對付這碑田之時，未透例遠，上前發出一個踉蹌，從石頂扑到，對林立時舉起銀針，叫道：“九陽神功大為奇少空刺便行分捷。
